In [14]:
import numpy as np
import cv2
import imutils
import pandas as pd
import time
import matplotlib.pyplot as plt 
import pytesseract
from numpy import unravel_index

In [15]:

def resize_image(img):
    img = cv2.resize(img, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
    return img

In [16]:
def show_image(img):
    cv2.imshow("image window",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 'Imaged displayed successfully'

In [53]:
def detecting_licenseplate(image):
    image = imutils.resize(image, width=500)
#     show_image(image)    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
#     show_image(gray)
    
    edged = cv2.Canny(gray, 170, 200)
    # ret,thresh = cv2.threshold(edged.copy(),127,255,0)
    #print(len(cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)))

    cnts,hierarchy = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#     show_image(img)
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
    NumberPlateCnt = None
    count = 0
    print(len(cnts))
#     for c in cnts:
# #             show_image(c)
#             peri = cv2.arcLength(c, True)
#             approx = cv2.approxPolyDP(c, 0.02 * peri, True)
#             mask = np.zeros(gray.shape,np.uint8)
#             newww_image = cv2.drawContours(mask,[approx],0,255,-1)
#             show_image(newww_image)
# #             show_image(approx)
#             if len(approx) == 4:
#                 NumberPlateCnt = approx
#                 print(NumberPlateCnt.shape)
#                 break
    peri = cv2.arcLength(cnts[1], True)
    approx = cv2.approxPolyDP(cnts[1], 0.02 * peri, True)
    NumberPlateCnt = approx
    print("orignal cnts ",NumberPlateCnt)
    ymax = np.max(NumberPlateCnt.reshape(4,2)[:,1])
    ymin = np.min(NumberPlateCnt.reshape(4,2)[:,1])
    xmax = np.max(NumberPlateCnt.reshape(4,2)[:,0])
    xmin = np.min(NumberPlateCnt.reshape(4,2)[:,0])
#     NumberPlateCnt[ymax] = NumberPlateCnt[ymax]+5
    ymax_indx  = index=np.where(NumberPlateCnt[:,:,:]==ymax)
    print("ymax_indx ",ymax_indx)
    ymax_indx = ymax_indx[0][0],ymax_indx[1][0],ymax_indx[2][0]
    xmax_indx  = index=np.where(NumberPlateCnt[:,:,:]==xmax)
    xmax_indx = xmax_indx[0][0],xmax_indx[1][0],xmax_indx[2][0]
    ymin_indx  = index=np.where(NumberPlateCnt[:,:,:]==ymin)
    ymin_indx = ymin_indx[0][0],ymin_indx[1][0],ymin_indx[2][0]
    xmin_indx  = index=np.where(NumberPlateCnt[:,:,:]==xmin)
    xmin_indx = xmin_indx[0][0],xmin_indx[1][0],xmin_indx[2][0]
    NumberPlateCnt[ymax_indx] = NumberPlateCnt[ymax_indx]+10
    NumberPlateCnt[xmax_indx] = NumberPlateCnt[xmax_indx]+10
    NumberPlateCnt[ymin_indx] = NumberPlateCnt[ymin_indx]-10
    NumberPlateCnt[xmin_indx] = NumberPlateCnt[xmin_indx]-10
    print("changed snts ",NumberPlateCnt)
    print(NumberPlateCnt)
    
#     Masking the part other than the number plate
    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[NumberPlateCnt],0,255,-1)
    show_image(new_image)
    new_image = cv2.bitwise_and(image,image,mask=mask)
#     show_image(new_image)
#     cv2.namedWindow("Final_image",cv2.WINDOW_NORMAL)
    #croping numberplate 
#     show_image(new_image)
#     new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
#     new_image = cv2.threshold(new_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    img_numberplate = np.where(new_image>0.8)
    last_index, = img_numberplate[0].shape
    x1,y1 = img_numberplate[1].min(),img_numberplate[0].min()
    x2,y2 = img_numberplate[1].max(),img_numberplate[0].max()
#     print("x1 = "+str(x1)+"x2 = "+str(x2)+"y1 = "+str(y1)+"y2 = "+str(y2))
    new_image = new_image[y1:y2,x1:x2]
#     show_image(new_image)
    return new_image#image[y1-10:y2+10,x1-10:x2+10]

In [18]:
def image_cleaning(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=500)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.GaussianBlur(img, (3, 3), 1)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return img

In [19]:
def reading_licenseplate(img):
    config = ('-l eng --oem 1 --psm 3')
    license_number = pytesseract.image_to_string(img, config=config)
    return license_number

In [20]:
def saving_image(img,name):
    cv2.imwrite(name,img)
    print('Imaged saved.')
    return True

In [73]:
def rotation(img):
    rows,cols = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-27,1)  #rotate image by some degree
    img = cv2.warpAffine(img,M,(cols,rows))
    return img

In [22]:
# C:\Users\KARAN KALRA\PycharmProjects\FirstProject\Yolo\alot_number.jpg

In [81]:
path_of_image = r'G:\LPR Project\car images\number-plate-design-for-38.jpg'
license_number = ""
# try:
img = cv2.imread(path_of_image)
print(type(img))
img = detecting_licenseplate(img)
# show_image(img)
# show_image(img)
# cv2.imwrite('CARRY.jpg',img)
img = resize_image(img)
img = image_cleaning(img)
# img = rotation(img)
# show_image(img)
license_number = reading_licenseplate(img)
# except Exception as err:
#     print(err)
show_image(img)
print(license_number)

<class 'numpy.ndarray'>
30
orignal cnts  [[[332 506]]

 [[328 524]]

 [[179 527]]

 [[176 507]]]
\KL 38 F 4004)
